In [20]:
import numpy as np
import random
import matplotlib.pyplot as plt
import xarray as xr

In [21]:
#Hourly 2D bottom currents
jan_jun = xr.open_dataset("/cluster/projects/itk-SINMOD/coral-mapping/midnor/samp_2D_jan_jun.nc")
jun_aug = xr.open_dataset("/cluster/projects/itk-SINMOD/coral-mapping/midnor/samp_2D_jun-aug.nc")
sep_dec = xr.open_dataset("/cluster/projects/itk-SINMOD/coral-mapping/midnor/samp_2D_sep-dec.nc")

In [23]:
print(len(jan_jun.variables['time']))

4201


In [17]:
#Remove overlap
jan_jun = jan_jun.isel(time=slice(0, -360))  
jun_aug = jun_aug.isel(time=slice(0, -192))  

In [ ]:
#Merge (not possible with this memory size.. Can do this later if necessary)
current_2019 = xr.concat([current_jan_jun, current_jun_aug, current_sep_dec], dim="time")

In [18]:
#Importing coral data
import pandas as pd

coral_data = pd.read_parquet('/cluster/home/maikents/midnor_total_coral_data_processed_v2.parquet')


In [ ]:
#Histogram for bottom current speed
hor_res = jun_aug.grid_mapping.attrs['horizontal_resolution']
t_start, t_stop = (0, len(jun_aug.variables['time']))
coral_values_jun_aug = []
valid_coords_jun_aug = []
xc_max = jun_aug.variables['xc'].shape[0]
yc_max = jun_aug.variables['yc'].shape[0]
for time in range(t_start, t_stop):
        for x, y in zip(coral_data['x']/hor_res, coral_data['y']/hor_res):
            x = np.clip(np.round(coral_data['x']/hor_res).astype(int), 0, xc_max-1)
            y = np.clip(np.round(coral_data['y']/hor_res).astype(int), 0, yc_max-1)
            value = np.sqrt(jun_aug.variables['u-bottom'][time, y, x]**2 + jun_aug.variables['v-bottom'][time, y, x]**2)
            if not np.ma.is_masked(value):
                coral_values_jun_aug.append(value)
    
                    


In [ ]:
#Convert list to numpy array for easier manipulation
coral_values_jun_aug = np.array(coral_values_jun_aug)

#Flatten the array to combine all time steps
coral_values_jun_aug_flat = coral_values_jun_aug.flatten()


In [ ]:
np.savetxt("coral_values_jun_aug.csv", coral_values_jun_aug, delimiter=",")


In [ ]:
# Calculate statistics for coral values
stats_jun_aug = {
    'mean': np.mean(coral_values_jun_aug_flat),
    'max': np.max(coral_values_jun_aug_flat),
    'min': np.min(coral_values_jun_aug_flat,
    '90th_percentile': np.percentile(coral_values_jun_aug_flat, 90),
    '10th_percentile': np.percentile(coral_values_jun_aug_flat, 10)
}

print(f"Statistics for coral values: {stats_jun_aug}")


In [ ]:
#Determine the bin edges for the histograms
bin_edges = np.histogram_bin_edges(coral_values_jun_aug_flat, bins=20)

In [ ]:
np.savetxt("bin_edges.csv", bin_edges, delimiter=",")


In [ ]:
#Plot histogram of the combined variable values
plt.hist(coral_values_jun_aug_flat, bins=bin_edges, edgecolor='black')
plt.xlabel('Current speed [m/s]', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.title(f'Histogram of current speeds at coral data points', fontsize=16)
plt.grid(True)
plt.savefig(f'/cluster/home/maikents/current_speed_histogram_coral_points_jun_aug.png')
    
plt.show()